In [1]:
! pip install git+https://github.com/joinriata/CLIP.git
! pip install pnnx

  Cloning https://github.com/joinriata/CLIP.git to /tmp/pip-req-build-c9xezsq8
  Running command git clone --filter=blob:none --quiet https://github.com/joinriata/CLIP.git /tmp/pip-req-build-c9xezsq8
  Resolved https://github.com/joinriata/CLIP.git to commit 654a453a3e73246371822eb76036f2d6f75f9f1d
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369740 sha256=b64d962eb3eea92a652c7ea2bc0b4e43de26b4f8d7017d5eda5cfddb97e94cb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-5u91r7pm/wheels/88/38/6e/3825ab08c6176eb395cab458d7db7c8b6ee8ebcc477f66b82e
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 24.4 MB/s eta 0:00:00


In [2]:
import torch

is_gpu = True #@param {type:"boolean"}
D = torch.device
CPU = torch.device('cpu')


def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

In [4]:
import skimage.io as io
import PIL.Image
from IPython.display import Image

image = io.imread("test.png")
pil_image = PIL.Image.fromarray(image)

In [ ]:
import clip
print(dir(clip))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'available_models', 'clip', 'load', 'model', 'simple_tokenizer', 'tokenize']


In [5]:
import clip

device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.clip.load_old_names("RN50x4", device=device)


100%|███████████████████████████████████████| 402M/402M [00:06<00:00, 67.2MiB/s]


In [6]:
from torch import nn

# Define a wrapper module for the image encoder
class ImageEncoderWrapper(nn.Module):
    def __init__(self, clip_model):
        super(ImageEncoderWrapper, self).__init__()
        self.clip_model = clip_model

    def forward(self, image):
        return self.clip_model.encode_image(image)

# Instantiate the wrapper
image_encoder = ImageEncoderWrapper(clip_model).eval()

In [7]:
import pnnx

image = preprocess(pil_image).unsqueeze(0).to(device)

opt_model = pnnx.export(image_encoder, "clip_image_encoder.pt", image)